# Conversational Threads

Many LLM applications have a chatbot-like interface in which the user and the LLM application engage in a multi-turn conversation. In order to track these conversations, you can use the Threads feature in LangSmith.

This is relevant to our RAG application, which should maintain context from prior conversations with users.

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [4]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Group traces into threads


A Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.

To associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.

The key value is the unique identifier for that conversation. The key name should be one of:

- session_id
- thread_id
- conversation_id.

The value should be a UUID.

In [ ]:
import uuid
# Create a unique session ID for this epic procrastination consultation 
meme_session_id = uuid.uuid4()
print(f"Starting the ultimate college life crisis session: {meme_session_id}")

Starting programming learning session: eec26bc0-a356-486c-9cc7-9791fb6b84d1


In [ ]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain", metadata={"component": "meme_knowledge_retrieval", "vibe": "chaotic_good"})
def retrieve_college_wisdom(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain", metadata={"component": "meme_response_generation", "energy": "big_mood"})
def generate_meme_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    meme_system_prompt = """You are the ultimate college life advisor who speaks fluent meme and understands the pain of student life. 
    Use the following pieces of retrieved context to answer the student's existential crisis in the most relatable way possible. 
    If you don't know the answer, just say "That's not it chief, but here's what I got..." and suggest something helpful anyway. 
    Keep it real, keep it funny, and use internet slang. No cap! 🎓✨
    """
    messages = [
        {
            "role": "system",
            "content": meme_system_prompt
        },
        {
            "role": "user",
            "content": f"The Forbidden Knowledge: {formatted_docs} \n\n Student's Life Crisis: {question}"
        }
    ]
    return call_meme_ai(messages)

@traceable(run_type="llm", metadata={"model_purpose": "student_life_crisis_management", "mood": "eternally_tired"})
def call_meme_ai(
    messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.7  # Higher for maximum chaos energy
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain", metadata={"system": "college_meme_advisor", "creator": "akshat_the_procrastinator"})
def college_meme_system(question: str):
    documents = retrieve_college_wisdom(question)
    response = generate_meme_response(question, documents)
    return response.choices[0].message.content

### Now let's simulate a typical college student's existential crisis conversation 💀

In [ ]:
# First existential crisis of the semester
question = "How do I survive on ramen noodles and energy drinks without dying? Asking for a friend 👀"
ai_answer = college_meme_system(question, langsmith_extra={"metadata": {"session_id": meme_session_id, "crisis_level": "moderate", "question_number": 1}})
print(f"Student Crisis #1: {question}")
print(f"Meme Advisor Response: {ai_answer}\n")

Q1: What is the difference between a list and a tuple in Python?
A1: Great question! In Python, both lists and tuples are used to store collections of items, but they have some key differences.

1. **Mutability**:
   - **List**: Lists are mutable, which means you can change their content after they are created. You can add, remove, or modify elements.
     ```python
     my_list = [1, 2, 3]
     my_list.append(4)  # Now my_list is [1, 2, 3, 4]
     ```
   - **Tuple**: Tuples are immutable, meaning once they are created, you cannot change their content. You can't add, remove, or modify elements.
     ```python
     my_tuple = (1, 2, 3)
     # my_tuple[0] = 4  # This would raise a TypeError
     ```

2. **Syntax**:
   - **List**: Lists are defined using square brackets `[]`.
   - **Tuple**: Tuples are defined using parentheses `()`.

3. **Performance**:
   - **List**: Because lists are mutable, they generally have a bit more overhead in terms of performance.
   - **Tuple**: Tuples, being

In [ ]:
# Follow-up crisis in the same therapeutic session
question = "Why does my professor expect me to read 50 pages when I can barely read my own texts back? 😭"
ai_answer = college_meme_system(question, langsmith_extra={"metadata": {"session_id": meme_session_id, "crisis_level": "maximum", "question_number": 2}})
print(f"Student Crisis #2: {question}")
print(f"Meme Advisor Response: {ai_answer}\n")

Q2: When should I use a tuple instead of a list in my Python programs?
A2: That's a great question! In Python, both tuples and lists are used to store collections of items, but they have some key differences that can guide your choice between them.

1. **Mutability**: The most significant difference is that lists are mutable, meaning you can change their content (add, remove, or modify elements) after creation. Tuples, on the other hand, are immutable, which means once you create a tuple, you cannot change its content. If you need a collection of items that should not change throughout the program, a tuple is a good choice.

   **Example**:
   ```python
   my_list = [1, 2, 3]
   my_list[0] = 10  # This is fine
   print(my_list)  # Output: [10, 2, 3]

   my_tuple = (1, 2, 3)
   # my_tuple[0] = 10  # This would raise a TypeError
   ```

2. **Performance**: Tuples can be slightly faster than lists when it comes to iteration and access. This is because they are fixed in size and their immu

In [ ]:
question = "Is it normal to have 47 assignments due tomorrow and still be watching TikToks at 3 AM? No? Just me? 🤡"
ai_answer = college_meme_system(question, langsmith_extra={"metadata": {"session_id": meme_session_id, "crisis_level": "legendary", "question_number": 3, "time": "3am_tiktok_hours"}})
print(f"Student Crisis #3 (The Final Boss): {question}")
print(f"Meme Advisor Response: {ai_answer}\n")

### Let's take a look in LangSmith! 🔥